# CNT Glyph Raytracing — Interactive Studio (SDF Ray Marching)

This notebook gives you a **live control panel** to ray‑march CNT glyphs using **signed distance fields** (SDFs).  
Tweak camera, quality, glyph mix, lighting, and shading (AO / soft shadows), then click **Render**.

### Quick start
1. **Run all cells** (Kernel → Run All).
2. Use the **control panel** to adjust settings; hit **Render**.
3. For speed, begin with **Preview** quality. Move to **Balanced** or **Studio** after framing your shot.

**Tip:** This is CPU‑only and optimized to be responsive. Your RTX 4070 can be used later by porting distance loops to CuPy/PyTorch.

In [11]:
import os, time, pathlib

print("Notebook CWD:", os.getcwd())

# List the most recent files so your render bubbles to the top
files = sorted((pathlib.Path('.').glob('*.*')), key=lambda p: p.stat().st_mtime, reverse=True)
for p in files[:20]:
    t = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(p.stat().st_mtime))
    print(f"{t}  {p.resolve()}")


Notebook CWD: C:\Users\caleb\cnt_genome
2025-10-02 07:51:14  C:\Users\caleb\cnt_genome\CNT_Glyph_Raytracing_Interactive.ipynb
2025-10-02 07:38:00  C:\Users\caleb\cnt_genome\CNT_3D_GenomicField_clean.ipynb
2025-10-02 07:37:13  C:\Users\caleb\cnt_genome\.ipynb_checkpoints
2025-10-02 07:36:47  C:\Users\caleb\cnt_genome\CNT_genomic_modules.csv
2025-10-02 07:36:46  C:\Users\caleb\cnt_genome\CNT_genomic_network_3D.html
2025-10-02 07:36:46  C:\Users\caleb\cnt_genome\CNT_genomic_network_3D.png
2025-10-02 07:35:24  C:\Users\caleb\cnt_genome\CNT_Glyph_Raytracing.ipynb
2025-10-02 07:31:57  C:\Users\caleb\cnt_genome\glyph_raytrace_preview.png
2025-10-02 07:30:55  C:\Users\caleb\cnt_genome\Untitled3.ipynb
2025-10-02 07:30:09  C:\Users\caleb\cnt_genome\CNT_one_field_atlas.html
2025-10-02 07:30:09  C:\Users\caleb\cnt_genome\CNT_one_field_atlas.png
2025-10-02 07:30:07  C:\Users\caleb\cnt_genome\CNT_module_gallery.html
2025-10-02 07:30:07  C:\Users\caleb\cnt_genome\CNT_module_index.csv
2025-10-02 07:30

In [12]:
# Core engine: SDFs, glyphs, scene, camera, and ray marcher
import numpy as np, matplotlib.pyplot as plt, time

# ---------------- Utilities ----------------
def norm(v): 
    n=np.linalg.norm(v, axis=-1, keepdims=True)
    return v/(n+1e-8)

def smin(a,b,k=0.18):
    h=np.clip(0.5+0.5*(b-a)/k,0,1); return (b*(1-h)+a*h)-k*h*(1-h)

def rot(p, axis='z', a=0.0):
    c,s=np.cos(a),np.sin(a)
    if axis=='z':
        R=np.array([[c,-s,0],[s,c,0],[0,0,1]])
    elif axis=='y':
        R=np.array([[ c,0,s],[0,1,0],[-s,0,c]])
    elif axis=='x':
        R=np.array([[1,0,0],[0,c,-s],[0,s,c]])
    else:
        R=np.eye(3)
    return p @ R.T

# ---------------- SDF primitives ----------------
def sdf_sphere(p,r): 
    return np.linalg.norm(p,axis=-1)-r

def sdf_torus(p,R=0.95,r=0.05):
    q=np.stack([np.linalg.norm(p[...,[0,2]],axis=-1)-R,p[...,1]],axis=-1)
    return np.linalg.norm(q,axis=-1)-r

def sdf_capsule(p,a,b,r):
    pa,ba=p-a,b-a
    h=np.clip((pa*ba).sum(-1)/(ba*ba).sum(-1),0.0,1.0)
    return np.linalg.norm(pa-ba*h[...,None],axis=-1)-r

def sdf_box(p,b):
    q=np.abs(p)-b
    return np.minimum(np.maximum(q[...,0],np.maximum(q[...,1],q[...,2])),0.0)+np.linalg.norm(np.maximum(q,0.0),axis=-1)

def sdf_plane(p,h=0.9): 
    return p[...,1]+h

# ---------------- CNT glyphs ----------------
def glyph_anchor(p, R=0.95, r=0.05, shaft=0.055, node=0.11, bar_w=0.40, bar_h=0.05):
    d=sdf_torus(p,R,r)
    d=smin(d,sdf_capsule(p,np.array([0,-0.52,0]),np.array([0,0.52,0]),shaft))
    d=smin(d,sdf_box(rot(p, 'z', 0.0)+np.array([0,-0.1,0]),np.array([bar_w,bar_h,0.06])))
    d=smin(d,sdf_sphere(p+np.array([0,0.25,0]),node))
    return d

def glyph_observer_ring(p, R=1.00, r=0.07, marks=3, mark_r=0.09):
    d=sdf_torus(p,R,r)
    for k in range(marks):
        ang=k*(2*np.pi/marks)
        c=np.array([np.cos(ang),0.0,np.sin(ang)])
        d=smin(d, sdf_sphere(p-0.95*c, mark_r), 0.15)
    return d

def glyph_collapse_benediction(p):
    p2=rot(p,'y',0.35)
    d=sdf_torus(p2+np.array([0, 0.35,0]),0.65,0.06)
    d=smin(d,sdf_torus(p2+np.array([0, 0.00,0]),0.85,0.08),0.15)
    d=smin(d,sdf_torus(p2+np.array([0,-0.35,0]),1.05,0.10),0.15)
    d=smin(d,sdf_capsule(p,np.array([0,-0.8,0]),np.array([0,0.8,0]),0.05),0.12)
    return d

# ---------------- Scene builder ----------------
def scene_distance(p, cfg):
    # p: (...,3)
    d = np.full(p.shape[:-1], 1e6, dtype=np.float32)
    if cfg['use_anchor']:
        d = np.minimum(d, glyph_anchor(p))
    if cfg['use_observer']:
        d = np.minimum(d, glyph_observer_ring(p - np.array([cfg['obs_x'],0.0,0.0]), R=cfg['obs_R'], r=cfg['obs_r'], marks=cfg['obs_marks'], mark_r=cfg['obs_mark_r']))
    if cfg['use_benediction']:
        d = np.minimum(d, glyph_collapse_benediction(rot(p + np.array([cfg['ben_x'],0.0,0.0]), 'y', cfg['ben_rot'])))
    if cfg['ground']:
        d = np.minimum(d, sdf_plane(p, h=cfg['ground_h']))
    return d

def estimate_normal(p, cfg, eps=1e-3):
    ex=np.array([eps,0,0]); ey=np.array([0,eps,0]); ez=np.array([0,0,eps])
    d0=scene_distance(p, cfg)
    n=np.stack([scene_distance(p+ex,cfg)-d0, scene_distance(p+ey,cfg)-d0, scene_distance(p+ez,cfg)-d0], axis=-1)
    nlen=np.linalg.norm(n)+1e-8
    return n/nlen

def soft_shadow(p, ldir, cfg, tmin=0.02, tmax=10.0, k=12.0):
    if not cfg['soft_shadows']:
        return 1.0
    res=1.0; t=tmin
    for _ in range(cfg['shadow_steps']):
        d=scene_distance(p+ldir*t, cfg)
        if d<1e-4: return 0.0
        res=min(res, k*d/t)
        t+=np.clip(d, 0.02, 0.5)
        if t>tmax: break
    return float(np.clip(res,0.0,1.0))

def ambient_occlusion(p, n, cfg, steps=3, step_size=0.08):
    if not cfg['ambient_occlusion']:
        return 1.0
    ao=0.0; w=1.0
    for i in range(steps):
        d=scene_distance(p+n*(i+1)*step_size, cfg)
        ao += w * ( (i+1)*step_size - d )
        w *= 0.6
    return float(np.clip(1.0 - 1.5*ao, 0.0, 1.0))

def ray_march(ro, rd, cfg):
    t=0.0
    for _ in range(cfg['steps']):
        p=ro+rd*t
        d=scene_distance(p, cfg)
        if d<cfg['eps']:
            return t, True
        t += d
        if t > cfg['far']:
            break
    return t, False

def camera_rays(W,H,cam):
    eye,look,up=cam['eye'],cam['look'],cam['up']
    fov=np.deg2rad(cam['fov']); aspect=W/H
    fwd=(look-eye); fwd=fwd/(np.linalg.norm(fwd)+1e-8)
    right=np.cross(fwd,up); right=right/(np.linalg.norm(right)+1e-8)
    upv=np.cross(right,fwd); upv=upv/(np.linalg.norm(upv)+1e-8)
    tan=np.tan(fov*0.5)
    xs=np.linspace(-tan*aspect,tan*aspect,W); ys=np.linspace(-tan,tan,H)
    xv,yv=np.meshgrid(xs,ys)
    dirs=(fwd[None,None,:]+xv[...,None]*right[None,None,:]+yv[...,None]*upv[None,None,:]).reshape(-1,3)
    dirs=dirs/(np.linalg.norm(dirs,axis=-1,keepdims=True)+1e-8)
    origins=np.repeat(eye[None,:],W*H,axis=0)
    return origins.reshape(H,W,3),dirs.reshape(H,W,3)

def render(cfg, save_path=None, ax=None):
    t0=time.time()
    W,H=cfg['W'],cfg['H']
    CAM=cfg['CAM']; LIGHT=cfg['LIGHT']
    orig,dirs=camera_rays(W,H,CAM)
    img=np.zeros((H,W,3),dtype=np.float32)
    for y in range(H):
        for x in range(W):
            ro,rd=orig[y,x],dirs[y,x]
            t,hit=ray_march(ro,rd,cfg)
            if not hit:
                img[y,x]=cfg['BG']; 
                continue
            p=ro+rd*t
            n=estimate_normal(p, cfg, eps=cfg['eps'])
            # heuristic: near plane distance implies ground
            is_ground = abs(scene_distance(p, cfg)-(p[1]+cfg['ground_h']))<1e-3 if cfg['ground'] else False
            albedo = cfg['albedo_ground'] if is_ground else cfg['albedo_glyph']
            # single light
            ldir = CAM['light_dir'] if cfg['use_dir_light'] else (LIGHT['pos']-p); ldir = ldir/(np.linalg.norm(ldir)+1e-8)
            ndotl = max(float(np.dot(n,ldir)), 0.0)
            sh = soft_shadow(p+n*0.002, ldir, cfg)
            ao = ambient_occlusion(p, n, cfg, steps=cfg['ao_steps'], step_size=cfg['ao_step'])
            col = albedo*(0.12*ao) + LIGHT['col']*albedo*ndotl*sh
            # emissive halos around glyphs
            gdist = glyph_anchor(p) if cfg['use_anchor'] else 1e6
            col = np.clip(col + cfg['glow']*np.exp(-max(gdist,0)*6.0)*cfg['glow_tint'], 0, 1)
            img[y,x]=col
    img = img ** (1.0/cfg['gamma'])
    if ax is None:
        plt.figure(figsize=(cfg['fig_w'],cfg['fig_h'])); plt.imshow(img); plt.axis('off'); plt.tight_layout()
        if save_path: plt.savefig(save_path, dpi=cfg['dpi'], bbox_inches='tight', pad_inches=0.0)
        plt.show()
    else:
        ax.clear(); ax.imshow(img); ax.axis('off')
    dt=time.time()-t0
    return img, dt
print("Engine ready.")

Engine ready.


In [13]:
import os, time
outdir = os.path.join(os.getcwd(), "renders")
os.makedirs(outdir, exist_ok=True)
stamp = time.strftime("%Y%m%d_%H%M%S")
base = save_name.value or "render.png"
name, ext = os.path.splitext(base)
save_full = os.path.join(outdir, f"{name}_{stamp}{ext or '.png'}")


In [14]:
# Interactive controls with ipywidgets
import ipywidgets as W
from IPython.display import display, clear_output

# --- Defaults / Presets ---
def make_cfg(quality='preview'):
    if quality=='preview':
        Wpx,Hpx,steps,far = 128,72,28,6.0
    elif quality=='balanced':
        Wpx,Hpx,steps,far = 360,202,60,8.0
    else: # studio
        Wpx,Hpx,steps,far = 720,405,120,12.0
    return {
        'W':Wpx, 'H':Hpx, 'steps':steps, 'far':far, 'eps':1e-3,
        'gamma':2.2, 'glow':0.4,
        'glow_tint':np.array([0.8,0.7,1.0])*0.15,
        'BG':np.array([0.03,0.03,0.04]),
        'albedo_glyph':np.array([0.85,0.82,1.0]),
        'albedo_ground':np.array([0.12,0.12,0.13]),
        'ground':True, 'ground_h':0.9,
        'ambient_occlusion':False, 'ao_steps':3, 'ao_step':0.08,
        'soft_shadows':False, 'shadow_steps':18,
        'use_anchor':True, 'use_observer':False, 'use_benediction':False,
        'obs_x':1.6, 'obs_R':1.0, 'obs_r':0.07, 'obs_marks':3, 'obs_mark_r':0.09,
        'ben_x':-1.6, 'ben_rot':0.45,
        'fig_w':6, 'fig_h':3.6, 'dpi':160,
        'use_dir_light':False,
        'LIGHT': {'pos':np.array([2.0,2.2,1.4]), 'col':np.array([1.0,0.95,0.9])*1.6},
        'CAM': {'eye':np.array([0.0,0.1,2.4]), 'look':np.array([0.0,0.0,0.0]), 'up':np.array([0.0,1.0,0.0]), 'fov':55.0, 'light_dir':np.array([0.6,0.7,0.4])}
    }

cfg = make_cfg('preview')

# --- Widgets ---
quality = W.ToggleButtons(options=['preview','balanced','studio'], value='preview', description='Quality')
use_anchor = W.Checkbox(True, description='Anchor')
use_observer = W.Checkbox(False, description='Observer Ring')
use_bened = W.Checkbox(False, description='Benediction')
obs_x = W.FloatSlider(1.6, min=0.6, max=3.0, step=0.1, description='Observer X')
obs_R = W.FloatSlider(1.0, min=0.6, max=1.6, step=0.05, description='Observer R')
obs_r = W.FloatSlider(0.07, min=0.02, max=0.20, step=0.01, description='Observer r')
obs_marks = W.IntSlider(3, min=0, max=8, step=1, description='Marks')
obs_mark_r = W.FloatSlider(0.09, min=0.03, max=0.2, step=0.01, description='Mark r')
ben_x = W.FloatSlider(-1.6, min=-3.0, max=-0.6, step=0.1, description='Bened X')
ben_rot = W.FloatSlider(0.45, min=-1.57, max=1.57, step=0.05, description='Bened Rot')

cam_eye_x = W.FloatSlider(0.0, min=-3, max=3, step=0.1, description='Eye X')
cam_eye_y = W.FloatSlider(0.1, min=-2, max=2, step=0.05, description='Eye Y')
cam_eye_z = W.FloatSlider(2.4, min=0.8, max=6, step=0.1, description='Eye Z')
cam_look_z = W.FloatSlider(0.0, min=-1, max=1, step=0.05, description='Look Z')
fov = W.FloatSlider(55.0, min=20, max=90, step=1, description='FOV')
use_dir_light = W.Checkbox(False, description='Dir Light')
light_x = W.FloatSlider(2.0, min=-6, max=6, step=0.2, description='Light X')
light_y = W.FloatSlider(2.2, min=-6, max=6, step=0.2, description='Light Y')
light_z = W.FloatSlider(1.4, min=-6, max=6, step=0.2, description='Light Z')

ao = W.Checkbox(False, description='Ambient Occlusion')
ao_steps = W.IntSlider(3, min=1, max=8, step=1, description='AO Steps')
ao_step = W.FloatSlider(0.08, min=0.02, max=0.2, step=0.01, description='AO Step')

shadows = W.Checkbox(False, description='Soft Shadows')
shadow_steps = W.IntSlider(18, min=4, max=64, step=2, description='Shadow Steps')

save_name = W.Text(value='render.png', description='Save As')
render_btn = W.Button(description='Render', button_style='primary')
out = W.Output()

# Layout groups
row1 = W.HBox([quality, use_anchor, use_observer, use_bened])
row2 = W.HBox([obs_x, obs_R, obs_r, obs_marks, obs_mark_r])
row3 = W.HBox([ben_x, ben_rot])
row4 = W.HBox([cam_eye_x, cam_eye_y, cam_eye_z, cam_look_z, fov])
row5 = W.HBox([use_dir_light, light_x, light_y, light_z])
row6 = W.HBox([ao, ao_steps, ao_step, shadows, shadow_steps, save_name, render_btn])

ui = W.VBox([row1, row2, row3, row4, row5, row6, out])
display(ui)

def on_render(_):
    render_btn.disabled=True
    try:
        c = make_cfg(quality.value)
        c['use_anchor']=use_anchor.value
        c['use_observer']=use_observer.value
        c['use_benediction']=use_bened.value
        c['obs_x']=obs_x.value; c['obs_R']=obs_R.value; c['obs_r']=obs_r.value
        c['obs_marks']=obs_marks.value; c['obs_mark_r']=obs_mark_r.value
        c['ben_x']=ben_x.value; c['ben_rot']=ben_rot.value
        c['CAM']['eye']=np.array([cam_eye_x.value, cam_eye_y.value, cam_eye_z.value])
        c['CAM']['look']=np.array([0.0, 0.0, cam_look_z.value])
        c['CAM']['fov']=fov.value
        c['use_dir_light']=use_dir_light.value
        c['ambient_occlusion']=ao.value; c['ao_steps']=ao_steps.value; c['ao_step']=ao_step.value
        c['soft_shadows']=shadows.value; c['shadow_steps']=shadow_steps.value
        c['LIGHT']['pos']=np.array([light_x.value, light_y.value, light_z.value])

        with out:
            clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(c['fig_w'], c['fig_h']))
            img, dt = render(c, save_path=save_name.value, ax=ax)
            fig.canvas.draw()
            print(f"Rendered {c['W']}x{c['H']} in {dt:.2f}s → saved as {save_name.value}")
    finally:
        render_btn.disabled=False

render_btn.on_click(on_render)
print("Controls ready. Adjust and press Render.")

Controls ready. Adjust and press Render.


## Notes & Next Steps

- For **faster previews**, stick to `preview` while framing, then switch to `balanced` or `studio`.
- Turn on **Ambient Occlusion** and **Soft Shadows** last—they cost the most.
- Want **animation** (e.g., breathing radius, camera orbit)? We can add a small loop that saves a numbered PNG sequence.

**CUDA plan:** Move `scene_distance`, `ray_march`, and `estimate_normal` to CuPy for 10–50× speedups on the RTX 4070.

In [15]:
import os
target = "Observer Ring.png".lower()
hits = []
for root, _, files in os.walk('.'):
    for f in files:
        if f.lower() == target:
            hits.append(os.path.abspath(os.path.join(root, f)))
print("Matches:", *hits, sep="\n- ")


Matches:
